In [11]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier,OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LinearRegression, LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier


In [12]:
training_data = pd.read_csv('train.csv')
training_data.columns

Index(['Won_Championship', 'Previous_SB_Wins', 'Number_Of_Wins_This_Season',
       'Number_Of_First_Round_Draft_Picks', 'Team_Value', 'Playing_Style',
       'Average_Player_Age', 'Number_Of_Injured_Players',
       'Coach_Experience_Level', 'ID'],
      dtype='object')

In [13]:
le_Team_Value = LabelEncoder()
training_data['Team_Value'] = le_Team_Value.fit_transform(training_data['Team_Value'])

le_Playing_Style = LabelEncoder()
training_data['Playing_Style'] = le_Playing_Style.fit_transform(training_data['Playing_Style'])

le_Number_Of_Injured_Players = LabelEncoder()
training_data['Number_Of_Injured_Players'] = le_Number_Of_Injured_Players.fit_transform(training_data['Number_Of_Injured_Players'])

le_Coach_Experience_Level = LabelEncoder()
training_data['Coach_Experience_Level'] = le_Coach_Experience_Level.fit_transform(training_data['Coach_Experience_Level'])

training_data = training_data.drop(columns=['ID'],axis=1)
training_data.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level
0,0,3,13,2,1,2,27,1,2
1,0,2,14,2,1,1,26,1,2
2,1,2,13,1,1,0,27,2,2
3,0,2,12,2,0,2,27,6,2
4,0,1,15,2,1,2,26,1,2


### Dropping columns -- ['Won_Championship','Team_Value','Playing_Style','Coach_Experience_Level']

In [14]:
y = training_data.Won_Championship
training_data = training_data.drop(columns=['Won_Championship','Team_Value','Playing_Style','Coach_Experience_Level'],axis=1)
#training_data = training_data.drop(columns=['Won_Championship','Coach_Experience_Level','Number_Of_Wins_This_Season','Number_Of_Injured_Players'],axis=1)

In [15]:
x_train,x_test, y_train, y_test = train_test_split(training_data,y,test_size=0.2)

In [16]:
bag = BaggingClassifier(n_estimators=100,oob_score=True,bootstrap_features=True)
bag.fit(x_train,y_train)
#bag.fit(training_data,y)
prediction = bag.predict(x_test)
acc = 100 * (f1_score(y_test,prediction,average='binary'))
acc

76.21905476369092

In [17]:
cols = training_data.columns
test_data = pd.read_csv('test.csv')
event_id = test_data['ID']

print(test_data.shape)
test_data = test_data.drop(columns=['ID','Team_Value','Playing_Style','Coach_Experience_Level'],axis=1)

#test_data['Team_Value'] = le_Team_Value.fit_transform(test_data['Team_Value'])

#test_data['Playing_Style'] = le_Playing_Style.fit_transform(test_data['Playing_Style'])

test_data['Number_Of_Injured_Players'] = le_Number_Of_Injured_Players.fit_transform(test_data['Number_Of_Injured_Players'])

#test_data['Coach_Experience_Level'] = le_Coach_Experience_Level.fit_transform(test_data['Coach_Experience_Level'])


predictions = bag.predict(test_data)
result_df = pd.DataFrame({'ID':event_id,'Won_Championship':predictions})
result_df.to_csv('Prediction.csv',index=False)

(3500, 9)


#### Online ACCURACY - 74.85